![](image.jpg)


Dive into the heart of data science with a project that combines healthcare insights and predictive analytics. As a Data Scientist at a top Health Insurance company, you have the opportunity to predict customer healthcare costs using the power of machine learning. Your insights will help tailor services and guide customers in planning their healthcare expenses more effectively.

## Dataset Summary

Meet your primary tool: the `insurance.csv` dataset. Packed with information on health insurance customers, this dataset is your key to unlocking patterns in healthcare costs. Here's what you need to know about the data you'll be working with:

## insurance.csv
| Column    | Data Type | Description                                                      |
|-----------|-----------|------------------------------------------------------------------|
| `age`       | int       | Age of the primary beneficiary.                                  |
| `sex`       | object    | Gender of the insurance contractor (male or female).             |
| `bmi`       | float     | Body mass index, a key indicator of body fat based on height and weight. |
| `children`  | int       | Number of dependents covered by the insurance plan.              |
| `smoker`    | object    | Indicates whether the beneficiary smokes (yes or no).            |
| `region`    | object    | The beneficiary's residential area in the US, divided into four regions. |
| `charges`   | float     | Individual medical costs billed by health insurance.             |



A bit of data cleaning is key to ensure the dataset is ready for modeling. Once your model is built using the `insurance.csv` dataset, the next step is to apply it to the `validation_dataset.csv`. This new dataset, similar to your training data minus the `charges` column, tests your model's accuracy and real-world utility by predicting costs for new customers.

## Let's Get Started!

This project is your playground for applying data science in a meaningful way, offering insights that have real-world applications. Ready to explore the data and uncover insights that could revolutionize healthcare planning? Let's begin this exciting journey!

In [65]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

# Loading the insurance dataset
insurance_data_path = 'insurance.csv'
insurance = pd.read_csv(insurance_data_path)
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19.0,female,27.900,0.0,yes,southwest,16884.924
1,18.0,male,33.770,1.0,no,Southeast,1725.5523
2,28.0,male,33.000,3.0,no,southeast,$4449.462
3,33.0,male,22.705,0.0,no,northwest,$21984.47061
4,32.0,male,28.880,0.0,no,northwest,$3866.8552


In [66]:
def clean_data(insurance):
    
    #Define Generic Categories for "sex"
    gender = {'M':'male', 'man':'male', 'F':'female', 'woman':'female'}
    insurance['sex'] = insurance['sex'].replace(gender)
    
    #Lowercase all region values
    insurance['region'] = insurance['region'].str.lower()

    #Format "charges" to float values removing any symbols
    insurance['charges'] = insurance['charges'].replace({'\$':''},regex=True).astype(float)

    #Remove Negative "age" values
    insurance = insurance[insurance["age"] > 0]

    #Assume "children" as 0 for any negative value
    insurance.loc[insurance['children'] < 0, 'children'] = 0

    #Drop the remaining missing data
    return insurance.dropna()
    

In [67]:
#Clean the data & check for missing data
cleaned_data = clean_data(insurance)
cleaned_data.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [68]:
#Select features & target variable
X = cleaned_data.drop('charges', axis=1)
y = cleaned_data['charges']

categorical_features = ['sex', 'smoker', 'region']
numerical_features = ['age', 'bmi', 'children']

#Encode categorical Data with dummy variables
X_cat_dummies = pd.get_dummies(X[categorical_features], drop_first=True)

#Scale numerical variables to format data for the model
X_processed = pd.concat([X[numerical_features], X_cat_dummies], axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_processed)

In [69]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

#Linear Regression Model
model = LinearRegression()

#Model Pipeline
steps = [("scaler", scaler), ("lin_reg", model)]
insurance_model_pipeline = Pipeline(steps)

#Fitting the Model
insurance_model_pipeline.fit(X_scaled, y)

from sklearn.model_selection import KFold

#Using Cross Validation to perform a more comprehensive evaluation
kfold = KFold(n_splits=6, random_state=42, shuffle=True)
mse_scores = -cross_val_score(insurance_model_pipeline, X_scaled, y, cv=kfold, scoring='neg_mean_squared_error')
r2_scores = cross_val_score(insurance_model_pipeline, X_scaled, y, cv=kfold, scoring='r2')

In [70]:
#Evaluate the Model
mean_mse = np.mean(mse_scores)
r2_score = np.mean(r2_scores)

print("Mean MSE:", mean_mse)
print("Mean R2:", r2_score)

Mean MSE: 37610898.3161387
Mean R2: 0.7406694193456588


In [71]:
#Read the validation data for predictions
validation_data = pd.read_csv('validation_dataset.csv')

#Encoding any categorical data in the validation dataset
validation_data_processed = pd.get_dummies(validation_data, columns=["sex", "smoker", "region"], drop_first=True)

#Predict charges
validation_predictions = insurance_model_pipeline.predict(validation_data_processed)

#Save prediction
validation_data["predicted_charges"] = validation_predictions

#Set Minimum Basic Charge for any values < 1000
validation_data.loc[validation_data["predicted_charges"] < 1000, "predicted_charges"] = 1000

validation_data.head()

,age,sex,bmi,children,smoker,region,predicted_charges
0,18.0,female,24.090000,1.0,no,southeast,128624.195643
1,39.0,male,26.410000,0.0,yes,northeast,220740.537449
2,27.0,male,29.150000,0.0,yes,southeast,181357.588606
3,71.0,male,65.502135,13.0,yes,southeast,423490.687270
4,28.0,male,38.060000,0.0,no,southeast,193247.431989
